## Model Training

### 1.1 Import Data and Required Packages
Importing Pandas, Numpy, Matplotlib, Seaborn, and Warnings Library.



In [167]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
# Suppress warnings
warnings.filterwarnings('ignore')

### Import the CSV Data as Pandas DataFrame

In [168]:
df = pd.read_csv('data/raw.csv')

In [169]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


###  Preparing X and Y Variables

In [170]:
X = df.drop(columns = ['math score'], axis = 1)

In [171]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [172]:
print("Categories in gender variable:      ",end = " ")
print(df['gender'].unique())

print("categories in race/ethinicity variable:     ", end = " ")
print(df['race/ethnicity'].unique())

print("Categories in parental level of eductation variable:     ", end = "  ")
print(df['parental level of education']. unique())

print("categories in lunch variable:         ", end = "  ")
print(df['lunch'].unique())

print("Categories in test preparation course variable:     ", end = "  ")
print(df['test preparation course'].unique())

Categories in gender variable:       ['female' 'male']
categories in race/ethinicity variable:      ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in parental level of eductation variable:       ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
categories in lunch variable:           ['standard' 'free/reduced']
Categories in test preparation course variable:       ['none' 'completed']


In [173]:
y = df['math score']

In [174]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [175]:
num_feature = X.select_dtypes(exclude = "object").columns
cat_feature = X.select_dtypes(include = "object").columns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_feature),
        ("StandardScaler", numeric_transformer, num_feature)
    ]
)

In [176]:
X = preprocessor.fit_transform(X)

In [177]:
X.shape

(1000, 19)

In [178]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

### Create an Evaluate Function to Provide All Metrics After Model Training

In [179]:
def evaluate_model(true, predicted):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    return mse, r2, mae

In [180]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')
    

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.8804
- Mean Absolute Error: 29.0952
- R2 Score: 4.2148
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8804
- Mean Absolute Error: 29.0952
- R2 Score: 4.2148


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.8253
- Mean Absolute Error: 42.5064
- R2 Score: 5.1579
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8253
- Mean Absolute Error: 42.5064
- R2 Score: 5.1579


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.8806
- Mean Absolute Error: 29.0563
- R2 Score: 4.2111
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.8806
- Mean Absolute Error: 29.0563
- R2 Score: 4.2111


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 0.7835
- Mean Absolute Error: 52.6834
- R2 Score: 5.6370
----------------

### Results

In [181]:
pd.DataFrame(list(zip(model_list, r2_list)), columns = ['Model Name', 'R2 Score']).sort_values(by = 'R2 Score', ascending = False)

,Model Name,R2 Score
4,Decision Tree,6.195000
3,K-Neighbors Regressor,5.637000
1,Lasso,5.157882
6,XGBRegressor,5.057731
8,AdaBoost Regressor,4.741226
5,Random Forest Regressor,4.658738
7,CatBoosting Regressor,4.612532
0,Linear Regression,4.214763
2,Ridge,4.211101
